# Segmenting and Clustering Neighborhoods in Toronto

In [1]:
! pip install lxml html5lib beautifulsoup4

In [2]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.10

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [3]:
import pandas as pd
import numpy as np

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

## 1. Download data and clean it up

### Download Toronto postal code data from Wikipedia

In [4]:
# Download Toronto postal code data
!wget -q -O 'canada_postal_codes_m.htm' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
print('Data downloaded!')

Data downloaded!


### Read data from html file. The dataframe has 3 columns: PostalCode, Borough, and Neighborhood

In [5]:
df = pd.read_html('canada_postal_codes_m.htm')[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [6]:
# Rename columns
df = df.rename(columns={'Postcode': 'PostalCode', 'Borough': 'Borough', 'Neighbourhood': 'Neighborhood'})
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Ignore cells with a borough that is 'Not assigned'

In [7]:
# Check
df[df['Borough'] == 'Not assigned'].shape

(77, 3)

In [8]:
# Remove them
df = df[df['Borough'] != 'Not assigned'].reset_index(drop=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [9]:
# Verify
df[df['Borough'] == 'Not assigned']

,PostalCode,Borough,Neighborhood


### Combine neighbourhoods that have same postal code

In [10]:
df = df.groupby(['PostalCode', 'Borough']).agg({'Neighborhood': lambda x: ', '.join(x)}).reset_index()
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### If neighborhood is not assigned, it will be the same as the borough

In [11]:
# Check
df[df['Neighborhood'] == 'Not assigned'].shape

(1, 3)

In [12]:
# Set values to them
df['Neighborhood'] = df['Neighborhood'].where(df['Neighborhood'] != 'Not assigned', df['Borough'])

In [13]:
# Verify
df[df['Neighborhood'] == 'Not assigned']

,PostalCode,Borough,Neighborhood


### Print the number of rows

In [14]:
df.shape

(103, 3)